In [9]:
from kss import split_sentences
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# ✅ 스타일 목록 및 매핑
styles = [
    'formal', 'informal', 'android', 'enfp', 'gentle', 'halbae', 'king', 'sosim', 'translator'
]

style_kor_map = {
    'formal': '문어체', 'informal': '구어체', 'android': '안드로이드',
    'enfp': 'enfp', 'gentle': '신사', 'halbae': '할아버지',
    'king': '왕', 'sosim': '소심한', 'translator': '번역기'
}

# ✅ 스타일별 max_length 설정
style_max_lengths = {
    'formal': 64, 'informal': 64, 'android': 32,
    'enfp': 80, 'gentle': 64, 'halbae': 64,
    'king': 72, 'sosim': 64, 'translator': 64
}

# ✅ 모델 로딩
MODEL_PATH = r"C:\Users\AI-LJH\Desktop\캡스톤\t5_style_fildata_finetuned\checkpoint-34040"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# ✅ 스타일 변환 함수 (개선 적용)
def style_transfer(sentence, target_style, max_length, num_beams=4):
    prompt = f"{target_style} 말투로 변환: {sentence}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=max_length).to(device)
    with torch.no_grad():
        output = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_length,
            num_beams=num_beams,
            no_repeat_ngram_size=3,
            repetition_penalty=1.2,
            early_stopping=False  # 더 자연스러운 완결성 확보
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)

# ✅ 긴 문장에 대해 전체 스타일 변환 수행
def convert_all_styles(text):
    sentences = split_sentences(text)
    results = {}

    for style in styles:
        styled_sentences = [
            style_transfer(s, style, max_length=style_max_lengths[style]) for s in sentences
        ]
        results[style] = "\n".join(styled_sentences)

    return results

# ✅ 테스트 문장
input_text = (
    "오늘은 정말 정신없는 하루였다. 회의가 계속 이어졌고, 중요한 보고서도 마무리해야 했다. "
    "지친 몸을 이끌고 집에 돌아오니 비로소 마음이 편안해졌다."
)

# ✅ 실행
outputs = convert_all_styles(input_text)

# ✅ 출력
for style in styles:
    print(f"🗂️ [{style}] {style_kor_map[style]}")
    print(outputs[style])
    print("-" * 80)


🗂️ [formal] 문어체
오늘 하루는 정말 정신없는 하루였어요. 이름은 어떻게 되나요? 어제 하루가 참 바빴는데. 잘 지내고 있으신가요? 감사합니다. 하루 종일 바쁘셨나요..? 저는 오늘 
회의가 계속 되면서 중요한 자료들을 마저 처리해야 했어요. 감사합니다. 서류로 작성해야 할 것도 있었어요... 마침 어제 회의를 마치고 돌아와서 다행이였던 내용들을 다 들고
네, 지쳐보이는 몸을 붙잡고 집에 돌아오니 마음이 평온해지는 것 같아요. 기분이 좋습니다. 그리고 다시 마음을 평온히 회복할 수 있었어요..! 마침 지금 퇴근하는 중
--------------------------------------------------------------------------------
🗂️ [informal] 구어체
오늘 하루는 정말 바쁜 하루였어. 나도 오늘 잘 지내고 있을까? 기분이 안 좋음. ...? 하루 종일 바빴어..!? 다음 날 너무 피곤해..,. 출근했어
회의가 계속 되면서 중요한 자료들을 마저 정리해야 했어...! 마침 오늘 회의까지 끝나서 다행이지. 서류 작성하는 시간이 너무 많아. 간신히 잡은 서류를 마저라도 써야 했어
응, 지쳐있던 몸을 붙잡고 집에 오니 마음이 후련해지는 것 같아. 기분이 괜찮아졌어..!. 마침 지금 퇴근하고 나서 다시 마음을 즐길 수 있었어. ...? 
--------------------------------------------------------------------------------
🗂️ [android] 안드로이드
오늘. 하루. 바쁨. 예상. 안된다. 안드로이드. 어제. 일함. 바쁜가. 오늘.
회의. 계속. 나옴. 중요한 자료. 마저 들음. 필요했음. 확인. 오늘. 회의
안드로이드. 약간. 지침. 그러나. 집 복귀. 마음. 평온해짐. 느낌. 또한. 듬
--------------------------------------------------------------------------------
🗂️ [enfp] enfp
오